In [ ]:
import agentpy as ap
import math

In [ ]:
class Sandpile(ap.Model):

    def setup(self):

        # Create agents (location for a pile).
        self.agents = ap.AgentList(self, self.p['size'])

        # Initialize avalanche size.
        self.num_pile_collapses = 0
        self.num_grains_moved = 0

        # Create grid (area) and place agents on them. Fill grid completely
        self.grid = ap.Grid(self, shape=[int(math.sqrt(self.p['size']))]*2, torus=True)
        self.grid.add_agents(self.agents, random=True, empty=True)

        # Agent properties: grains = 0; collapse = 0.
        self.agents.grains = 0
        self.agents.collapse = False

    def step(self):
        
        # Reset counters.
        self.num_pile_collapses = 0
        self.num_grains_moved = 0
        self.agents.collapse = False
        
        # Randomly select an agent on a grid and increment their grain
        # pile by 1.
        self.agents.random(n=1).grains += 1

        # Collapse agent grain piles that exceed grain limits. A collapse
        # involves the agent giving away one grain to four of six possible
        # neighbors. Keep collpasing agent grain piles until the grid does not
        # contain any agents exceeding grain limits.
        while any(self.agents.select(self.agents.grains > self.p['grain_limit'])):

            # Find the agents on the grid exceeding grain limits.
            agnts = self.agents.select(self.agents.grains > self.p['grain_limit'])

            # Size of the avalanche is equal to the number of collapsing piles.
            # Ultimately, the number of collapsing piles is equal to the number
            # of agents with grain piles exceeding the limit.

            # If a cell collapses more than once during a string of collapses,
            # then it is counted more than once. For example, X agent's pile
            # collapses, causing enough of their neighbors to collapse, that X
            # reaches the limit again and collapses.
            self.num_pile_collapses += len(agnts)

            # For each agent exceeding grain limits, select frpm their six
            # neighbors and them one grain.
            for agnt in agnts:
                agnt.collapse = True

                for neighbor in self.grid.neighbors(agnts).random(n=self.p['neighbors'], replace=False):
                    if agnt.grains > 0:
                        agnt.grains -= 1
                        neighbor.grains += 1
                        self.num_grain_moved += 1
                    
    def end(self):
        pass

In [ ]:
parameters = {
    'seed': 92,       # Seed for RNG.
    'steps': 10,      # Number of steps.
    'size': 9,        # Number of agents and ^2 grid size. Choose size with rational root.
    'grain_limit': 4, # Limit connected with neighbors.
    'neighbors': 4    # Max number of neighbors is 6.
 }


model = Sandpile(parameters)
results = model.run()
# results.save(exp_name=exp_name, path='data')